In [1]:

%matplotlib inline
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import FinanceDataReader as fdr

Using TensorFlow backend.


In [2]:
pandf = fdr.DataReader("KS11","2009.01.01","2019-01-01")
pandf

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2009-01-02,1157.40,1132.87,1157.40,1118.84,340780000.0,0.0293
2009-01-05,1173.57,1180.98,1185.22,1166.66,414760000.0,0.0140
2009-01-06,1194.28,1183.52,1203.59,1179.94,444200000.0,0.0176
2009-01-07,1228.17,1198.85,1228.56,1195.20,568560000.0,0.0284
2009-01-08,1205.70,1212.70,1218.43,1195.15,472340000.0,-0.0183
...,...,...,...,...,...,...
2018-12-21,2061.49,2052.70,2061.51,2049.76,311390000.0,0.0007
2018-12-24,2055.01,2050.38,2059.94,2046.18,285280000.0,-0.0031
2018-12-26,2028.01,2028.81,2037.83,2014.28,321500000.0,-0.0131


In [3]:
# convert nparray
nparr = pandf.values
nparr.astype('float32')
print(nparr)

[[ 1.15740e+03  1.13287e+03  1.15740e+03  1.11884e+03  3.40780e+08
   2.93000e-02]
 [ 1.17357e+03  1.18098e+03  1.18522e+03  1.16666e+03  4.14760e+08
   1.40000e-02]
 [ 1.19428e+03  1.18352e+03  1.20359e+03  1.17994e+03  4.44200e+08
   1.76000e-02]
 ...
 [ 2.02801e+03  2.02881e+03  2.03783e+03  2.01428e+03  3.21500e+08
  -1.31000e-02]
 [ 2.02844e+03  2.03209e+03  2.03557e+03  2.02139e+03  3.98020e+08
   2.00000e-04]
 [ 2.04104e+03  2.03670e+03  2.04697e+03  2.03541e+03  3.52680e+08
   6.20000e-03]]


In [4]:
# normalization
scaler = MinMaxScaler(feature_range=(0, 1))
nptf = scaler.fit_transform(nparr)
nptf

array([[0.08774962, 0.08707471, 0.08125887, 0.07909785, 0.2815341 ,
        0.75432811],
       [0.09798782, 0.11720825, 0.09888968, 0.10908167, 0.3426833 ,
        0.62819456],
       [0.11110056, 0.11879917, 0.11053159, 0.11740842, 0.36701734,
        0.65787304],
       ...,
       [0.63898492, 0.64824372, 0.63922759, 0.64055152, 0.26559795,
        0.40478153],
       [0.63925718, 0.65029814, 0.63779533, 0.64500959, 0.32884662,
        0.51442704],
       [0.64723499, 0.6531856 , 0.64502003, 0.65380033, 0.29137021,
        0.56389118]])

In [37]:
# split train, test

y=np.where(pandf["Change"].shift(-1).dropna()>0,1,0)
train_size = int(len(nptf) * 0.7)
y_train=y[:train_size]
y_test=y[train_size:]
X_train = nptf[:train_size]
X_test = nptf[train_size:]

In [38]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [53]:
# simple lstm network learning
model = Sequential()
model.add(LSTM(36, input_shape=(1, 6)))
for i in range(2):
    model.add(Dense(36,activation='tanh'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, batch_size=16,validation_split=0.3)

Epoch 1/100
1731/1731 [==============================] - 1s 691us/step - loss: 0.6940 - acc: 0.5165
Epoch 2/100
1731/1731 [==============================] - 0s 235us/step - loss: 0.6938 - acc: 0.5165
Epoch 3/100
1731/1731 [==============================] - 0s 222us/step - loss: 0.6922 - acc: 0.5095
Epoch 4/100
1731/1731 [==============================] - 0s 184us/step - loss: 0.6915 - acc: 0.5188
Epoch 5/100
1731/1731 [==============================] - 0s 156us/step - loss: 0.6934 - acc: 0.5026
Epoch 6/100
1731/1731 [==============================] - 0s 192us/step - loss: 0.6921 - acc: 0.5170
Epoch 7/100
1731/1731 [==============================] - 0s 225us/step - loss: 0.6925 - acc: 0.5090
Epoch 8/100
1731/1731 [==============================] - 0s 212us/step - loss: 0.6928 - acc: 0.5124
Epoch 9/100
1731/1731 [==============================] - 0s 205us/step - loss: 0.6914 - acc: 0.5188
Epoch 10/100
1731/1731 [==============================] - 0s 194us/step - loss: 0.6915 - acc: 0.5205

1731/1731 [==============================] - 0s 207us/step - loss: 0.6902 - acc: 0.5367
Epoch 82/100
1731/1731 [==============================] - 0s 181us/step - loss: 0.6899 - acc: 0.5309
Epoch 83/100
1731/1731 [==============================] - 0s 229us/step - loss: 0.6899 - acc: 0.5326
Epoch 84/100
1731/1731 [==============================] - 0s 218us/step - loss: 0.6905 - acc: 0.5240
Epoch 85/100
1731/1731 [==============================] - 0s 183us/step - loss: 0.6901 - acc: 0.5274
Epoch 86/100
1731/1731 [==============================] - 0s 184us/step - loss: 0.6900 - acc: 0.5292
Epoch 87/100
1731/1731 [==============================] - 0s 184us/step - loss: 0.6906 - acc: 0.5211
Epoch 88/100
1731/1731 [==============================] - 0s 180us/step - loss: 0.6899 - acc: 0.5436
Epoch 89/100
1731/1731 [==============================] - 0s 187us/step - loss: 0.6912 - acc: 0.5257
Epoch 90/100
1731/1731 [==============================] - 0s 174us/step - loss: 0.6904 - acc: 0.5234
Epo

In [54]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train,np.where(model.predict(X_train)>0.5,1,0))

0.5389948006932409